# Experiment 1: Typical use case
This is representative of a normal use-case on METASPACE, which makes it suitable for head-to-head comparisons. There is often limited time available on the higher-spec PC used for initial data capture as it is a shared resource, so usually the analysis will be performed from scientists’ or students’ lower-spec laptops.

### METRICS TO BENCHMARK
* Performance:
    * **Metric:** Total processing time up to downloading the results dataframe
    
        **Goal:** Faster than serverful METASPACE (including or excluding cluster start time)

    * **Metric:** Latency for retrieving all images of target ions.
    
        **Goal:** Similar to or faster than METASPACE’s python client

* Capability:
    * **Metric:** Peak memory usage on client.
    
        **Goal:** Capable of running on low-spec PC with 8GB ram, so ~6GB max usage

* Cost:
    * **Metric:** Cloud provider cost
    
        **Goal:** Similar price or cheaper than METASPACE (including or excluding cluster start time)

    * **Metric:** Developer time
    
        **Goal:** Less annual time required to manage cloud infrastructure than METASPACE

# Notebook setup
Run `python3 setup.py install` to install all requirements for annotation pipeline project.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%config Completer.use_jedi = False
%matplotlib inline

In [ ]:
# We need this to overcome Python notebooks limitations of too many open files
import resource
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print('Before:', soft, hard)

# Raising the soft limit. Hard limits can be raised only by sudo users
resource.setrlimit(resource.RLIMIT_NOFILE, (10000, hard))
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print('After:', soft, hard)

In [ ]:
# These are Python and Python lib path we want to use
import sys
sys.executable, sys.prefix

In [ ]:
# Install PyWren-IBM if needed
try:
    import pywren_ibm_cloud as pywren
except ModuleNotFoundError:    
    !{sys.executable} -m pip install -U pywren-ibm-cloud==1.0.10
    import pywren_ibm_cloud as pywren

pywren.__version__

In [ ]:
#Install psutil if needed
try:
    import psutil
except ModuleNotFoundError:    
    !{sys.executable} -m pip install -U psutil
    import psutil

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

## Configuration

In [ ]:
import json
config = json.load(open('config.json'))

In [ ]:
#input_config = json.load(open('metabolomics/input_config_small.json'))
input_config = json.load(open('metabolomics/input_config_big.json'))
#input_config = json.load(open('metabolomics/input_config_huge.json'))
input_data = input_config['dataset']
input_db = input_config['molecular_db']

# Override config to match METASPACE annotation settings
input_data['num_decoys'] = 20
input_db['modifiers'] = ['']

# Setup (Not included in benchmark timings)

In [ ]:
from annotation_pipeline.molecular_db import dump_mol_db, build_database, \
    calculate_centroids, get_formula_id_dfs, clean_formula_chunks

In [ ]:
# Download commonly used mol DBs from METASPACE (add force=True to redownload if needed)
dump_mol_db(config, config['storage']['db_bucket'], 'metabolomics/db/mol_db1.pickle', 22) #HMDB-v4
dump_mol_db(config, config['storage']['db_bucket'], 'metabolomics/db/mol_db2.pickle', 19) #ChEBI-2018-01
dump_mol_db(config, config['storage']['db_bucket'], 'metabolomics/db/mol_db3.pickle', 24) #LipidMaps-2017-12-12
dump_mol_db(config, config['storage']['db_bucket'], 'metabolomics/db/mol_db4.pickle', 26) #SwissLipids-2018-02-02

In [ ]:
num_formulas, formula_chunk_keys = build_database(config, input_db)

In [ ]:
polarity = input_data['polarity']
isocalc_sigma = input_data['isocalc_sigma']
centroids_shape, centroids_head = calculate_centroids(config, input_db, formula_chunk_keys, polarity, isocalc_sigma)

In [ ]:
clean_formula_chunks(config, input_db, formula_chunk_keys)

# Benchmark

In [ ]:
import os
import psutil # "pip install psutil" if needed
import pandas as pd
from datetime import datetime
memory_usage_mb = psutil.Process(os.getpid()).memory_info().rss / 2**20
print(f'Memory usage before: {memory_usage_mb:.0f}MB')

In [ ]:
from annotation_pipeline.utils import init_pywren_stats
init_pywren_stats()

### Run Annotation Pipeline

In [ ]:
from annotation_pipeline.pipeline import Pipeline

In [ ]:
pipeline = Pipeline(config, input_config)

In [ ]:
start_time = datetime.now()
pipeline()
results_df = pipeline.get_results()
finish_time = datetime.now()

In [ ]:
print('start', start_time)
print('finish', finish_time)
print('duration', finish_time - start_time)

In [ ]:
# Display PyWren statistics file
pd.read_csv('stats.csv')

In [ ]:
from annotation_pipeline.utils import remove_pywren_stats
remove_pywren_stats()

In [ ]:
memory_usage_mb = psutil.Process(os.getpid()).memory_info().rss / 2**20
print(f'Memory usage after: {memory_usage_mb:.0f}MB')

# Benchmark Part 2
Comparing the time required to retrieve all images

In [ ]:
#Install metaspace2020 if needed (May require kernel restart)
try:
    from metaspace.sm_annotation_utils import SMInstance
except ModuleNotFoundError:    
    !{sys.executable} -m pip install -U metaspace2020
    from metaspace.sm_annotation_utils import SMInstance

In [ ]:
%time images_dict = pipeline.get_images()

In [ ]:
sm = SMInstance()
# TODO: (EMBL) Reprocess these datasets with the other DBs, because currently they only have HMDB-v4
ds = sm.dataset(id='2016-09-22_11h16m11s') # For input_config_small.json
#ds = sm.dataset(id='2016-09-21_16h06m52s') # For input_config_big.json
#ds = sm.dataset(id='2016-09-21_16h06m49s') # For input_config_huge.json

In [ ]:
%time metaspace_client_images = ds.all_annotation_images(fdr=0.5)

# Check results are correct (WIP)

In [ ]:
results_df[lambda df: (df.mol == 'C42H84NO8P') & (df.adduct == '+H')]

In [ ]:
reference_results = ds.results()
reference_results = reference_results.rename({'moc':'chaos', 'rhoSpatial': 'spatial', 'rhoSpectral': 'spectral'}, axis=1)
reference_results = reference_results[['chaos', 'spatial', 'spectral', 'msm', 'fdr']]
reference_results = reference_results.reset_index()

filtered_results = results_df
filtered_results = filtered_results[filtered_results.database_path == 'metabolomics/db/mol_db1.pickle']
#filtered_results = filtered_results[filtered_results.fdr <= 0.5]
filtered_results = filtered_results[filtered_results.adduct != '']
filtered_results = filtered_results[filtered_results.modifier == '']
filtered_results = filtered_results.rename({'mol': 'formula'}, axis=1)
filtered_results = filtered_results[['chaos', 'spatial', 'spectral', 'msm', 'fdr', 'formula', 'adduct', 'modifier']]

merged_results = reference_results.merge(filtered_results, how='outer', left_on=['formula','adduct'], right_on=['formula','adduct'], suffixes=['_ref',''])
merged_results = merged_results.sort_values(['formula','adduct'])

In [ ]:
common_results[['spatial', 'spatial_ref']]

In [ ]:
common_results = merged_results[merged_results.fdr.notna() & merged_results.fdr_ref.notna()]
spatial_wrong = common_results[np.abs((common_results.spatial - common_results.spatial_ref)) >= 0.01]
assert len(spatial_wrong) < 5, spatial_wrong # Higher tolerance because older DSs seem to have used a different algorithm
spectral_wrong = common_results[np.abs((common_results.spectral - common_results.spectral_ref)) >= 0.001]
assert len(spectral_wrong) <= 1, spectral_wrong
chaos_wrong = common_results[np.abs((common_results.chaos - common_results.chaos_ref)) >= 0.01]
assert len(chaos_wrong) <= 1, chaos_wrong
msm_wrong = common_results[np.abs((common_results.msm - common_results.msm_ref)) >= 0.001]
assert len(msm_wrong) <= 10, msm_wrong

In [ ]:

def quantize_fdr(fdr):
    if fdr <= 0.05: return 1
    if fdr <= 0.1: return 2
    if fdr <= 0.2: return 3
    if fdr <= 0.5: return 4
    return 5
    
fdr_level = merged_results.fdr.apply(quantize_fdr)
fdr_ref_level = merged_results.fdr_ref.apply(quantize_fdr)

fdr_exact = merged_results[fdr_level == fdr_ref_level]
fdr_near = merged_results[np.abs(fdr_level - fdr_ref_level) <= 1]
print(len(fdr_exact), len(fdr_near), len(merged_results))
assert len(fdr_exact) >= len(merged_results) * 0.5
assert len(fdr_near) >= len(merged_results) * 0.8

# Clean Temp Data

In [ ]:
from annotation_pipeline.utils import clean_from_cos

In [ ]:
# Clean dataset chunks
clean_from_cos(config, config["storage"]["ds_bucket"], input_data["ds_chunks"])

In [ ]:
# Clean dataset segments
clean_from_cos(config, config["storage"]["ds_bucket"], input_data["ds_segments"])

In [ ]:
# Clean centroids database segments
clean_from_cos(config, config["storage"]["db_bucket"], input_db["centroids_segments"])

In [ ]:
# Clean formula output images
clean_from_cos(config, config["storage"]["output_bucket"], output["formula_images"])

In [ ]:
# Clean FDR rankings
clean_from_cos(config, config["storage"]["ds_bucket"], input_data["fdr_rankings"])